In [4]:
import numpy as np
from math import log10, sqrt 
import cv2
from numpy.random import uniform, normal, exponential, rayleigh
import matplotlib.pyplot as plt
from numpy.fft import fft2,ifft2


def salt_pepper_noise(f,probability):
    g = f.copy()
    nr,nc = f.shape[:2]
    for x in range(nr):
        for y in range(nc):
            value = uniform(0,1)
            if value > 0 and value <= probability / 2:
                g[x,y] = 0
            elif value > probability/2 and value <= probability:
                g[x,y] = 255
            else:
                g[x,y] = f[x,y]
    return g

def band_filtering(f,filter,D0,width,order):
    nr,nc = f.shape[:2]
    fp = np.zeros([nr,nc])
    for x in range(nr):
        for y in range(nc):
            fp[x,y]=pow(-1,x+y)*f[x,y]
    F = fft2(fp)
    G = F.copy()
    
    if filter==1:
        for u in range(nr):
            for v in range(nc):
                dist = np.sqrt((u-nr/2)*(u-nr/2)+(v-nc/2)*(v-nc/2))
                if dist>=D0 - width/2 and dist <= D0 + width/2:
                    G[u,v]=0
                    
    if filter==2:
        for u in range(nr):
            for v in range(nc):
                dist = np.sqrt((u-nr/2)*(u-nr/2)+(v-nc/2)*(v-nc/2))
                if dist>=D0 - width/2 and dist > D0 + width/2:
                    G[u,v]=0
                    
    gp = ifft2(G)
    gp2 = np.zeros([nr,nc])
    for x in range(nr):
        for y in range(nc):
            gp2[x,y] = round(pow(-1,x+y)*np.real(gp[x,y]),0)
    g = np.uint8(np.clip(gp2,0,255))
    
    return g
                    
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
    
    
def histogram(f):
    if f.ndim!=3:
        hist = cv2.calcHist([f],[0],None,[256],[0,256])
        plt.plot(hist)
    else:
        color = ('b','g','r')
        for i, col in enumerate(color):
            hist = cv2.calcHist(f,[i],None,[256],[0,256])
            plt.plot(hist,color=col)
    plt.xlim([0,256])
    plt.xlabel("Intensity")
    plt.ylabel("#Intensity")
    plt.show()

    

def main():
    img1 = cv2.imread("Lenna.bmp",-1)
    img2_1 = salt_pepper_noise(img1,0.05)
    img2_2 = band_filtering(img2_1,1,100,20.0,1)
    img3_1 = salt_pepper_noise(img1,0.15)
    img3_2 = band_filtering(img3_1,1,100,20.0,1)
    img4_1 = salt_pepper_noise(img1,0.30)
    img4_2 = band_filtering(img4_1,1,100,20.0,1)
    cv2.imshow("Original Image",img1)
    cv2.imshow("salt_pepper_noise Image 0.05",img2_2)
    cv2.imshow("salt_pepper_noise Image 0.15",img3_2)
    cv2.imshow("salt_pepper_noise Image 0.30",img4_2)
    print("IBRF:")
    print(f"salt_pepper_noise Image 0.05: {PSNR(img2_2,img1)}")
    print(f"salt_pepper_noise Image 0.05: {PSNR(img3_2,img1)}")
    print(f"salt_pepper_noise Image 0.05: {PSNR(img4_2,img1)}")
    #############################################################
    img2_2 = band_filtering(img2_1,2,100,20.0,1)
    img3_2 = band_filtering(img3_1,2,100,20.0,1)
    img4_2 = band_filtering(img4_1,2,100,20.0,1)
    print("IBPF:")
    print(f"salt_pepper_noise Image 0.05: {PSNR(img2_2,img1)}")
    print(f"salt_pepper_noise Image 0.05: {PSNR(img3_2,img1)}")
    print(f"salt_pepper_noise Image 0.05: {PSNR(img4_2,img1)}")
    
    cv2.waitKey(0)
main()

IBRF:
salt_pepper_noise Image 0.05: 31.19519670666969
salt_pepper_noise Image 0.05: 29.42222412831835
salt_pepper_noise Image 0.05: 28.663702329217568
IBPF:
salt_pepper_noise Image 0.05: 30.121533830412048
salt_pepper_noise Image 0.05: 28.677616335642515
salt_pepper_noise Image 0.05: 28.17246139895225
